In [13]:
import sys
sys.path.append('../src')
from decoder import Decoder
import torch
import numpy as np
import scanpy as sc
from sklearn.preprocessing import StandardScaler
import torch
import pytorch_lightning as pl
from torch.utils.data import DataLoader, TensorDataset
import wandb
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning.callbacks import ModelCheckpoint
import numpy as np
from sklearn.preprocessing import StandardScaler
import joblib
import os
import hydra
from omegaconf import DictConfig


In [6]:
adata = sc.read_h5ad('../data/rna_figure_ready.h5ad')

In [7]:
X_pca = adata.obsm['X_pca']

In [8]:
X_phate = adata.obsm['X_phate']

In [9]:
subsample_idx = np.random.choice(X_pca.shape[0], size=1000, replace=False)

In [10]:
x = X_pca[subsample_idx]
y = X_phate[subsample_idx]

In [11]:
x_scaler = StandardScaler()
y_scaler = StandardScaler()

x_scaled = x_scaler.fit_transform(x)
y_scaled = y_scaler.fit_transform(y)


In [15]:
dataset = TensorDataset(torch.FloatTensor(x_scaled), torch.FloatTensor(y_scaled))

In [21]:
train_ds, val_ds, test_ds = torch.utils.data.random_split(dataset, [.70, .15, .15])
train_loader = DataLoader(train_ds, batch_size=8, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=8, shuffle=True)
test_loader = DataLoader(test_ds, batch_size=8, shuffle=False)

In [22]:
wandb_logger = WandbLogger(project="cflows_decoder")
checkpoint_callback = ModelCheckpoint(dirpath="checkpoints", save_top_k=1, monitor="val_loss")

model = Decoder(input_dim=x.shape[1], output_dim=y.shape[1], layer_widths=[32, 32], dropout=0.1, batchnorm=True)


In [23]:
trainer = pl.Trainer(
    max_epochs=5,
    logger=wandb_logger,
    callbacks=[checkpoint_callback],
    log_every_n_steps=1
)

trainer.fit(model, train_loader, val_loader)
trainer.test(model, test_loader)

/gpfs/gibbs/pi/krishnaswamy_smita/xingzhi/.conda_envs/mioflow/lib/python3.10/site-packages/lightning_fabric/plugins/environments/slurm.py:204: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /gpfs/gibbs/pi/krishnaswamy_smita/xingzhi/.conda_env ...
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: xingzhis. Use `wandb login --relogin` to force relogin



  | Name    | Type             | Params | Mode 
-----------------------------------------------------
0 | net     | Sequential       | 2.9 K  | train
1 | loss_fn | CrossEntropyLoss | 0      | train
-----------------------------------------------------
2.9 K     Trainable params
0         Non-trainable params
2.9 K     Total params
0.012     Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/gpfs/gibbs/pi/krishnaswamy_smita/xingzhi/.conda_envs/mioflow/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:475: Your `val_dataloader`'s sampler has shuffling enabled, it is strongly recommended that you turn shuffling off for val/test dataloaders.


Training: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=5` reached.
/gpfs/gibbs/pi/krishnaswamy_smita/xingzhi/.conda_envs/mioflow/lib/python3.10/site-packages/lightning_fabric/plugins/environments/slurm.py:204: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /gpfs/gibbs/pi/krishnaswamy_smita/xingzhi/.conda_env ...


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_loss         │    -20.619455337524414    │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': -20.619455337524414}]

In [30]:
adata.uns['pca']['variance_ratio']

array([0.05846943, 0.02992528, 0.02476442, 0.02000062, 0.01521769,
       0.01307336, 0.0099673 , 0.00912144, 0.00798036, 0.00568012,
       0.00537586, 0.00402243, 0.00350719, 0.00284831, 0.00276139,
       0.00263608, 0.00226976, 0.00219904, 0.00208077, 0.00188534,
       0.00179846, 0.00170207, 0.00158183, 0.00152271, 0.00144567,
       0.00143286, 0.0013392 , 0.00131123, 0.00126212, 0.00119322,
       0.00113601, 0.00105825, 0.00103508, 0.00100976, 0.00096298,
       0.00096005, 0.00093657, 0.0009171 , 0.00089559, 0.00086381,
       0.00084855, 0.00083571, 0.00082911, 0.00082439, 0.00080758,
       0.00077505, 0.00074541, 0.00074371, 0.00072908, 0.00072562],
      dtype=float32)